<a href="https://colab.research.google.com/github/uhyozzy/Project/blob/master/Predicting%20Road%20Flooding%20Risk/Model/%EB%8B%A4%EB%B3%80%EC%9D%B8%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%AA%A8%EB%8D%B8_%EB%B0%8F_%EC%98%88%EC%B8%A1(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **■ 지역환경과 하수관로 비율 선형회귀 학습 후 예측**

# [1] TensorFlow를 이용한 **다변인 선형회귀 모델**

참고1 : https://www.youtube.com/watch?v=9Ur7bbZtoTc 혹은 https://ndb796.tistory.com/126

참고2 : https://youtu.be/ve6gtpZV83E

- 다변인 선형회귀는 모델에 영향을 미치는 변인이 여러 개 일 때 사용하는 모델임. 현재 우리의 데이터에서는 변인이 '강수량', '지형', '지역환경이므로 이 모든 변인이 '하수관로비율'에 영향을 미친다고 감안해야 함.
- 학습률을 0.000005로 설정하여 되도록 짧은 시간에 정확한 결과가 나오도록 구현

**Base**

In [1]:
import tensorflow as tf
import numpy as np
from pandas.io.parsers import read_csv

In [2]:
# 파일을 쉼표로 구분하여 읽어 데이터를 DataFrame 형태로 불러옴
data_origin = read_csv('/content/지역환경,하수관로.csv', sep=',')

In [3]:
# 결측치 확인
missing_values = data_origin.isnull().sum()
print(missing_values)

遺덊닾?섎㈃(%)                0
?뱀? 硫댁쟻??%)              0
?섏쿇 硫댁쟻??%)              0
蹂듦컻?섏쿇 媛쒖닔(媛?            0
留⑦? 媛쒖닔(媛?               0
鍮쀫Ъ諛쏆씠 媛쒖닔(媛?            0
鍮쀫Ъ ?뚰봽 媛쒖닔(媛?          30
[MAX] ?섏닔愿濡?鍮꾩쑉(%)    183
dtype: int64


In [4]:
# 결측치가 있는 행 제거한 후 확인
data = data_origin.dropna()
missing_values2 = data.isnull().sum()
print(missing_values2)

遺덊닾?섎㈃(%)              0
?뱀? 硫댁쟻??%)            0
?섏쿇 硫댁쟻??%)            0
蹂듦컻?섏쿇 媛쒖닔(媛?          0
留⑦? 媛쒖닔(媛?             0
鍮쀫Ъ諛쏆씠 媛쒖닔(媛?          0
鍮쀫Ъ ?뚰봽 媛쒖닔(媛?         0
[MAX] ?섏닔愿濡?鍮꾩쑉(%)    0
dtype: int64


In [5]:
# DataFrame을 NumPy 배열로 변환하고 데이터 타입을 float32로 지정
xy = np.array(data, dtype=np.float32)

In [6]:
# 7개의 변인을 입력을 받음
x_data = xy[:, 0:7]

In [7]:
# 하수관로(MAX) 값을 입력 받음
# 출력 데이터로 사용할 마지막 열의 데이터를 추출
y_data = xy[:, [-1]]

In [8]:
# 입력 데이터를 위한 TensorFlow placeholder를 생성합니다.
# 데이터 형태는 [None, 7]과 [None, 1]인데, None은 임의의 행 개수를 의미 -> X는 2개의 변인, y는 1개의 변인
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 7], name="X")
Y = tf.placeholder(tf.float32, shape=[None, 1], name="Y")

In [9]:
# 가중치 변수 W를 생성. 초기값은 정규 분포를 따르는 랜덤한 값임
W = tf.Variable(tf.random_normal([7, 1]), name="weight")

In [10]:
# 편향 변수 b를 생성
b = tf.Variable(tf.random_normal([1]), name="bias")

In [11]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [12]:
# 비용 함수를 설정합니다.(평균 제곱 오차를 계산 )
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [13]:
# 최적화 함수를 설정

# 경사 하강법 최적화기를 생성하고 학습률을 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
# 비용 함수를 최소화하기 위한 훈련 오퍼레이션을 생성
train = optimizer.minimize(cost)

In [14]:
# 세션을 생성
sess = tf.Session()

In [15]:
# 글로벌 변수를 초기화
sess.run(tf.global_variables_initializer())

**학습 수행**
- ※주의※ Nan값 있으면 학습이 제대로 수행안되니 결측치 필수로 제거 후 학습 진행

In [16]:
# 학습을 수행합니다.

for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
        print("#", step, " 손실 비용: ", cost_)
        print("- 하수관로비율(MAX): ", hypo_[0])

# 100001번 반복하는 학습 루프
# 학습을 수행하면서 비용, 가설 값, 훈련을 실행하여 비용 최소화를 진행
# 매 500번째 스텝마다 학습 상황에 대한 내용을 출력

# 0  손실 비용:  6.056206
- 하수관로비율(MAX):  [-1.3621731]
# 500  손실 비용:  5.993985
- 하수관로비율(MAX):  [-1.347848]
# 1000  손실 비용:  5.93269
- 하수관로비율(MAX):  [-1.3336785]
# 1500  손실 비용:  5.8723164
- 하수관로비율(MAX):  [-1.3196546]
# 2000  손실 비용:  5.812868
- 하수관로비율(MAX):  [-1.3057914]
# 2500  손실 비용:  5.754302
- 하수관로비율(MAX):  [-1.2920727]
# 3000  손실 비용:  5.6965995
- 하수관로비율(MAX):  [-1.2784932]
# 3500  손실 비용:  5.6397295
- 하수관로비율(MAX):  [-1.2650608]
# 4000  손실 비용:  5.583735
- 하수관로비율(MAX):  [-1.2517668]
# 4500  손실 비용:  5.528572
- 하수관로비율(MAX):  [-1.2386315]
# 5000  손실 비용:  5.4742136
- 하수관로비율(MAX):  [-1.2256199]
# 5500  손실 비용:  5.4206305
- 하수관로비율(MAX):  [-1.212752]
# 6000  손실 비용:  5.3678617
- 하수관로비율(MAX):  [-1.2000167]
# 6500  손실 비용:  5.3158665
- 하수관로비율(MAX):  [-1.1874287]
# 7000  손실 비용:  5.2646213
- 하수관로비율(MAX):  [-1.1749626]
# 7500  손실 비용:  5.2141023
- 하수관로비율(MAX):  [-1.1626326]
# 8000  손실 비용:  5.1643496
- 하수관로비율(MAX):  [-1.1504332]
# 8500  손실 비용:  5.115318
- 하수관로비율(MAX):  [-1.1383682]
# 9000  손실 비용:  5.066974


**학습 모델 저장**

In [17]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./saved.cpkt")
print('학습된 모델을 저장했습니다.')

# 모델 저장을 위한 Saver 객체를 생성
# 학습된 모델을 저장
# 저장 완료 메시지를 출력

학습된 모델을 저장했습니다.


# [2] 다변인 선형회귀를 활용한 **하수관로 비율 예측**

**Base**

In [18]:
import tensorflow as tf
import numpy as np

In [19]:
# 플레이스 홀더를 설정.
# 입력 데이터와 출력 데이터를 위한 TensorFlow placeholder를 생성.
# 입력 데이터는 [None, 7] 모양이고 출력 데이터는 [None, 1] 모양임.

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 7])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [20]:
#가중치와 편향 변수를 생성. 가중치의 모양은 [7, 1]이고 편향의 모양은 [1]

W = tf.Variable(tf.random_normal([7, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

In [21]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [22]:
# TensorFlow 그래프와 세션 설정
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

# 학습 모델 불러오기
save_path = "./saved.cpkt"
saver = tf.compat.v1.train.import_meta_graph(save_path + ".meta")
saver.restore(sess, save_path)

# 그래프로부터 텐서 가져오기
graph = tf.compat.v1.get_default_graph()
X = graph.get_tensor_by_name("X:0")  # Placeholder 이름 변경
hypothesis = graph.get_tensor_by_name("add:0")  # hypothesis 텐서 이름 변경


In [23]:
# 사용자로부터 강수량을 입력받습니다.

impervious = float(input('불투수면 (%) : '))
greenery = float(input('녹지 면적율(%) : '))
river = float(input('하천 면적율(%): '))
covered_river = float(input('복개하천 개수(개) : '))
manhole = float(input('맨홀 개수(개)) : '))
rain_gutter = float(input('빗물 1받이 개수(개) : '))
pump = float(input('빗물 펌프 개수(개) : '))

불투수면 (%) : 0.53
녹지 면적율(%) : 0.03
하천 면적율(%): 0.24
복개하천 개수(개) : 1
맨홀 개수(개)) : 12322
빗물 1받이 개수(개) : 24000
빗물 펌프 개수(개) : 0


In [24]:
# 입력 데이터를 준비
data = np.array([[impervious,greenery,river,covered_river,
                  manhole,rain_gutter,pump]], dtype=np.float32)

feed_dict = {X: data}
prediction = sess.run(hypothesis, feed_dict=feed_dict)

print("예측된 하수관로비율(MAX):", prediction[0][0])

예측된 하수관로비율(MAX): 2119.1353


# **예측 모델 평가**
- 평균 절대 오차(MAE)가 작을수록 모델의 예측이 실제 데이터와 더 가깝다는 의미
- 결정 계수(R-squared)가 1에 가까울수록 모델이 주어진 데이터를 잘 설명한다는 의미

In [25]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = sess.run(hypothesis, feed_dict={X: x_data})

mae = mean_absolute_error(y_data, y_pred)
r2 = r2_score(y_data, y_pred)

print("평균 절대 오차(MAE):", mae)
print("결정계수(R-squared):", r2)

평균 절대 오차(MAE): 0.805439
결정계수(R-squared): -0.7350343396156127


## **MSE**

In [26]:
# 예측 수행
feed_dict = {X: x_data}
y_pred = sess.run(hypothesis, feed_dict)

# Mean Squared Error 계산
mse = np.mean(np.square(y_data - y_pred))

# R-squared 계산
total_variance = np.var(y_data)
explained_variance = np.var(y_pred)
r2 = 1 - (mse / total_variance)

print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 1.699981
